In [1]:
import os
HOME = os.getcwd()
print(HOME)

d:\Sahayta-main\Sahayta-main\detectFloodVictims


In [2]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:

!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

In [ ]:
#custom data to train for YOLO model
!mkdir {HOME}/datasets
%cd {HOME}/datasets
!pip install roboflow


from roboflow import Roboflow
rf = Roboflow(api_key="ADD_ROBOFLOW_KEY")
project = rf.workspace("yolov5-zexzl").project("flood-person-detection-fxn2y")
version = project.version(2)
dataset = version.download("yolov8")




In [ ]:
%cd {HOME}
!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=10 imgsz=640 plots=True

In [ ]:
!ls {HOME}/runs/detect/train/

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/results.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/val_batch0_pred.jpg', width=600)


Validate Model

In [ ]:
%cd {HOME}
!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

Inference with custom Model

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/runs/detect/predict/*.jpg')[:3]:
      display(Image(filename=image_path, width=600))
      print("\n")

In [ ]:
from google.colab import files
import shutil

shutil.make_archive("runs", 'zip', "runs")
files.download("runs.zip")

In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key="ADD_ROBOFLOW_KEY")
project = rf.workspace("modellabel").project("yolo-floods-relief")
version = project.version(1)
version.deploy("yolov8", f"{HOME}/runs/detect/train/weights", "best.pt")

# project.version(dataset.version).deploy(model_type="yolov8", model_path=f"{HOME}/runs/detect/train/")

In [ ]:
#Run inference on your model on a persistant, auto-scaling, cloud API

#load model
model = project.version(dataset.version).model

#choose random test set image
import os, random
test_set_loc = dataset.location + "/test/images/"
random_test_image = random.choice(os.listdir(test_set_loc))
print("running inference on " + random_test_image)

pred = model.predict(test_set_loc + random_test_image, confidence=40, overlap=30).json()
pred